# Simiosis gia Grid
- o skopos tou na dimiourgisw ena grid einai gia na miosw kata poli to sinolo ton simion sto disance join
- auto epitinxanete ftiaxnontas ena MBB pernontas ta min ton max(x,y) kai apo ta 2 data sets kathos kai ta max apo ta min(x,y) me tin logiki oti osa simia einai ektos tou min max x enos apo ta 2 dataset den ikanopoioun tin sinthiki theta.
- stin sinexia tha prepei na kanw spatial join to grid kai me ta 2 datasets gia na vrw se poio grid paei kathe simio
- meta tha aporipsw osa grids den exoun kanena

### Xrisima stixia na periexei ena grid
- gridId
- polygon
- TODO Check id x an y needed for partitioning reasons

### Idea gia grid filtering
- Na kanw spatial join mono sto dataset me ta ligotera simia prokimenou na filtrarw ta grids pou den exeoun kanena simio apo to Dataset auto
- Me auton ton tropo epitinxanw na kanw mikrotero spatial join sto dataset me tis perisoteres eggrafes
- auto vasizete sto

# Simiosis gia partitioning
- prepei na xorisw ta data mou se ola ta mixanimata prin ksekinisw to spatial join
- episis prepei na xorisw kai ta grids omiomorfa se ola ta mixanimata
- enas tropos gia na ta xorisw tha mporouse na einai to X. Diladi na vrw ta max kai min X kai paota 2 datasets gia na vrw to rangekai na kanw range partition os pros x gia ta points
- gia ta grids apo tin all tha eprepe na kanw partition os pros to max X tous oste na simenei oti oti grid einai sto partition exei points mono sto partition auto.
- me auton ton tropo epitinxanoume na exoume ta points kai ta grids tous mirasmena ana mixanima kai na empodisoume tis metakinisis sto diktio

### Pws to kanw auto?
- prepei na dw pws douleui to pratitioning
- pos simperiferete to join sto partitioning
- prepei na dw an ipostirizei kati tetoio i vivliothiki

## Imports

In [5]:
# Main  Imports
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.core.display import display
import time

# Local App Imports
from utils import spatialUtils as utils

# Spark Imports
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DataType


# Sedona Imports
from sedona.utils import KryoSerializer, SedonaKryoRegistrator
from sedona.register import SedonaRegistrator
from sedona.sql.types import GeometryType

# Python shapely imports
import shapely.geometry as sg
from descartes import PolygonPatch
import geog
import shapely.speedups
shapely.speedups.enable() # this makes some spatial queries run faster

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

## Load Dataset

In [ ]:
# full program timer
total_time = time.time()

test_sample_A_path = "../testData/ais_one_hour.csv"
test_sample_B_path = "../testData/ais_one_hour2.csv"

# test_sample_path = "../testData/ais_one_month.csv"


# create sedona spark session
spark = SparkSession. \
    builder. \
    appName('appName'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,'
           'org.datasyslab:geotools-wrapper:geotools-24.0'). \
    getOrCreate()

SedonaRegistrator.registerAll(spark)

# Load CSV File
df_a = spark.read.csv(header="True", inferSchema="True", path=test_sample_A_path)
df_a.show(10)

df_b = spark.read.csv(header="True", inferSchema="True", path=test_sample_B_path)
df_b.show(10)

## Filter Dataset for Test Reasons
- arxika tha periorisoume to dataset stin perioxi x[-6.5,-5] kai y[47.5,49]
- epilegw auto gt kserw oti exei arketa pings

In [ ]:
df_a = df_a.filter( (-6 <= df_a.X) & (df_a.X <= -5) & (47.5 <= df_a.Y) & (df_a.Y <= 49))
print("Total filtered_df_a count {}".format(df_a.count()))
df_a.show(10)
print("Dataset A count: {}".format(df_a.count()))

df_b = df_b.filter( (-6.5 <= df_b.X) & (df_b.X <= -5) & (47.5 <= df_b.Y) & (df_b.Y <= 49))
print("Total filtered_df_a count {}".format(df_b.count()))
df_b.show(10)
print("Dataset B count: {}".format(df_b.count()))

## Grid Dataset
Lets create a test grid
- We have to finde the MBB of the dataset
- Then we have to create a grid for this dataset (**sos** each grid side has to be equal to theta)

In [ ]:
cell_time = time.time()

# find min/max X,Y from dataset A
mbb_row_a = df_a.agg(
    F.max(df_a.X),
    F.min(df_a.X),
    F.max(df_a.Y),
    F.min(df_a.Y),
).collect()[0]
print("print mbb: {}".format(mbb_row_a))

# find min/max X,Y from dataset B
mbb_row_b = df_b.agg(
    F.max(df_b.X),
    F.min(df_b.X),
    F.max(df_b.Y),
    F.min(df_b.Y),
).collect()[0]
print("print mbb: {}".format(mbb_row_b))

# find minimum bounding GRID box from 2 datasets
min_x = max(mbb_row_a["min(X)"], mbb_row_b["min(X)"])
max_x = min(mbb_row_a["max(X)"], mbb_row_b["max(X)"])
min_y = max(mbb_row_a["min(Y)"], mbb_row_b["min(Y)"])
max_y = min(mbb_row_a["max(Y)"], mbb_row_b["max(Y)"])

#calculate grid for dataset
grid = utils.getPolyGrid(min_x,min_y,max_x,max_y, theta=10)
print("grid len: {}".format(len(grid)))

print("--- %s seconds ---" % (time.time() - cell_time))


## Plot Grid And Points

In [ ]:
# print grids
ax = utils.createAXNFigure()
BLUE = '#6699cc'
GRAY = '#999999'
for cell in grid:
    ax.add_patch(PolygonPatch(cell["geometry"], fc=GRAY, ec=GRAY, alpha=0.5, zorder=2))

for row in df_a.collect():
  ax.plot(row.X, row.Y, marker='o', markersize=2, alpha=0.3, c='r')

for row in df_b.collect():
  ax.plot(row.X, row.Y, marker='x', markersize=2, alpha=0.3, c='b')


plt.ylabel("Latitude")
plt.xlabel("Longitude")
plt.show()

## Create Grid RDD or DF

In [ ]:
grid_df = spark.createDataFrame(grid)
grid_df.printSchema()
print(grid_df.schema)
grid_df.show(10)

gpd_grid_df = gpd.GeoDataFrame(grid)

## Link Points tou Grids
- **Approach 1:** Test databricks code for spatial join from link
- **Approach 2:** perform spatial join in order to link points with grids

### Approach 1

def find_intersection(longitude, latitude):
    # row = grid_df.collect()[0]
    # print(row)
    mgdf = gpd_grid_df.apply(lambda x: x.grid_id if x.geometry.intersects(sg.Point(longitude, latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return int(first_valid_value)

find_intersection_udf = F.udf(find_intersection, IntegerType())


cell_time = time.time()

joined = df_a.withColumn("cell_id", find_intersection_udf(df_a.X, df_a.Y))
joined.show(10)

print("--- %s seconds ---" % (time.time() - cell_time))

### Approach 2

In [ ]:
def createPoints(longitude, latitude):
  return sg.Point(longitude, latitude)

createPoints_udf = F.udf(createPoints,GeometryType())

In [ ]:
sqlContext = SQLContext(spark)
# timing procedure
start_time = time.time()

geom_a_df = df_a.withColumn("geometry", createPoints_udf(df_a.X, df_a.Y)).cache()

geom_a_df.createOrReplaceTempView("points")
grid_df.createOrReplaceTempView("grids")

spatial_join_result = spark.sql(
    """
        SELECT P._id, P.X, P.Y, P.geometry as point_geometry,  G.geometry as grid_geometry, G.grid_id
        FROM points AS P, grids AS G
        WHERE ST_Intersects(P.geometry, G.geometry)
    """
).cache() # cache results to avoid perform join again


spatial_join_result.show()
print("--- %s seconds ---" % (time.time() - start_time))

## Spatial join Approach 2 stats
- find count
- find pings per grid
- filter grids with 0 points

### SOS
Otan kanoume chaining functions mporoume na exoume access se colum apo to proigoumeno function kanontas F.col("coll_name")

In [ ]:
print("spatial join count: {}".format(spatial_join_result.count()))

# find pings per grid
filtered_grid = spatial_join_result.groupby(["grid_geometry", "grid_id"])\
    .count()\
    .filter(F.col("count") > 0)\
    .cache()

filtered_grid.show(10)

print("filtered spatial join count: {}".format(filtered_grid.count()))


In [ ]:
# print grids
ax = utils.createAXNFigure()
BLUE = '#6699cc'
GRAY = '#999999'
for row in filtered_grid.collect():
    ax.add_patch(PolygonPatch(row["grid_geometry"], fc=GRAY, ec=GRAY, alpha=0.5, zorder=2))

for row in geom_a_df.collect():
  ax.plot(row.X, row.Y, marker='o', markersize=2, alpha=0.3, c='r')

for row in df_b.collect():
  ax.plot(row.X, row.Y, marker='x', markersize=2, alpha=0.3, c='b')


plt.ylabel("Latitude")
plt.xlabel("Longitude")
plt.show()

## Spatial Join 2nd dataset
at this stap er perform spatial join for second dataset with filtered grid


In [ ]:
# timing procedure
start_time = time.time()

geom_b_df = df_b.withColumn("geometry", createPoints_udf(df_b.X, df_b.Y)).cache()
geom_b_df.createOrReplaceTempView("points_B")
filtered_grid.createOrReplaceTempView("grids")


spatial_join_b_result = spark.sql(
    """
        SELECT P._id, P.X, P.Y, P.geometry as point_geometry,  G.grid_geometry, G.grid_id
        FROM points_B AS P, grids AS G
        WHERE ST_Intersects(P.geometry, G.grid_geometry)
    """
).cache() # cache results to avoid perform join again


spatial_join_b_result.show()
print("--- %s seconds ---" % (time.time() - start_time))

# Calculate Expanded MBB's
- First we have to check the count of filtered girds
- Check if points exists with null grid
- Create a UDF that calculates Expanded MBB's
-

In [ ]:
print("Filter grid count: {}".format(filtered_grid.count()))
filtered_grid.show(10)

# spatial_join_b_result.select([F.count(F.when(F.isnan(c), c))
#                              .alias(c) for c in spatial_join_b_result.columns])\
#     .show()

print("B point Set count: {}".format(spatial_join_b_result.count()))
spatial_join_b_result.select("grid_id").\
    select([F.count(F.when(F.isnan(F.col("grid_id")), F.col("grid_id")))])\
    .show()


In [ ]:
print("--- TOTAL EXECUTION TIME ---\n--- %s seconds ---" % (time.time() - total_time))